In [ ]:
API = "http://localhost:8000"

def upload_ui(file):
    files = {"file": (file.name, open(file.name, "rb"))}
    r = requests.post(f"{API}/upload", files=files)
    if r.status_code == 200:
        return r.json()
    else:
        return {"error": f"Upload failed: {r.status_code} - {r.text}"}

def chat_ui(message, history):
    r = requests.post(f"{API}/chat", params={"q": message})
    if r.status_code == 200:
        data = r.json()
        reply = data["answer"]
        sources = ", ".join(data["sources"])
        if sources:
            reply += f" Sources: {sources}"
        history.append({"role": "user", "content": message})
        history.append({"role": "assistant", "content": reply})
        return history, history
    else:
        error_msg = f"Chat failed: {r.status_code} - {r.text}"
        history.append({"role": "user", "content": message})
        history.append({"role": "assistant", "content": error_msg})
        return history, history


def launch_ui():
    with gr.Blocks() as demo:
        gr.Markdown(" Smart Contract Assistant")

        with gr.Tab("Upload"):
            file = gr.File()
            out = gr.JSON()
            btn = gr.Button("Upload & Index")
            btn.click(upload_ui, file, out)

        with gr.Tab("Chat"):
            chatbot = gr.Chatbot()
            msg = gr.Textbox()
            msg.submit(chat_ui, [msg, chatbot], [chatbot, chatbot])

    demo.launch()